In [1]:
from custom_dataset import Dataset_Assay, Dataset_Assay_Survival, custom_collate_assay, custom_collate_assay_survival
from models import *
from saveAndLoad import *

# LOAD DATA
data_dir = '../labeled_data/'
labeled_data = os.listdir(data_dir)
for ni,i in sorted(zip(labeled_data,range(len(labeled_data)))):print(i,'\t',ni)
data_emb = labeled_data[23]
print(data_emb)
data_df_emb = pd.read_csv(data_dir+data_emb)

nlabels = len(data_df_emb['int_label'].unique())
print('n labels:',nlabels)
device = 'cuda:0'

mut_embeddings = np.load('../aa/canonical_mut_average_embeddings_esm2.npy')
ref_embeddings = np.load('../aa/canonical_ref_embeddings_esm2.npy')
tumors = pickleLoad('../aa/tumors.pkl')
assays = pickleLoad('../aa/assays.pkl')

/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


37 	 BINARYdata_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
24 	 BINARYdata_CANCER_TYPE_0MinMutations_169MinCancerType.csv
1 	 BINARYdata_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
32 	 BINARYdata_CANCER_TYPE_3MinMutations_169MinCancerType.csv
12 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
26 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
10 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_1696MinCancerType.csv
30 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_169MinCancerType.csv
23 	 data_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
25 	 data_CANCER_TYPE_0MinMutations_169MinCancerType.csv
41 	 data_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
17 	 data_CANCER_TYPE_3MinMutations_169MinCancerType.csv
11 	 data_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
28 	 data_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
4 	 data_CANCER_TYPE_DETAILED_3MinMutations_1696MinCancerType.csv
47 	 data_CANCER_TYPE_DETAIL

In [2]:
## HYPERPARAMS
num_epochs = 10
learning_rate = 0.001

In [3]:
dataset = Dataset_Assay(data_df_emb, mut_embeddings, ref_embeddings, tumors, assays, device)
train_loader, test_loader = getTrainTestLoaders(dataset, batch_size = 500, collate=custom_collate_assay)

138303 samples
124702 unique patients
20 labels


In [4]:
## ATTENTION MODEL WITH POSITION EMBEDDING
class Config_Att:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    max_len : int = 1448
    position_embedding: bool = False
    num_heads: int = 8

config_att = Config_Att()
config_att.n_labels = nlabels

model = Classifier(config_att)
model.to(device)

num_epochs = 10
train_assay(model,num_epochs,train_loader,test_loader)

TRAINING:  16%|█▌        | 36/222 [03:47<19:33,  6.31s/it, Epoch=1/5, Loss: 2.1311]


KeyboardInterrupt: 

In [ ]:
# dataset = Dataset_Assay_Survival(data_df_emb, mut_embeddings, ref_embeddings, tumors, assays, device)
# train_loader, test_loader = getTrainTestLoaders(dataset, batch_size = 500, collate=custom_collate_assay_survival)

In [ ]:
# ## ATTENTION MODEL WITH POSITION EMBEDDING
# class Config_Att:
#     n_layer: int = 3
#     input_dim: int = 640
#     dropout: float = 0.0
#     bias: bool = False
#     n_labels: int = 1
#     pooling : str = 'mean'
#     norm_fn: nn.Module = nn.LayerNorm
#     max_len : int = 1448
#     position_embedding: bool = False

# config_att = Config_Att()

# model = Classifier(config_att)
# model.to(device)

# num_epochs = 15
# optimizer = optim.Adam(model.parameters(), lr=.00001)
# criterion = negative_log_partial_likelihood
# train_assay_survival(model,num_epochs,train_loader,test_loader, criterion, optimizer)